In [3]:
# ====================================================================
# NOTEBOOK 01: CONSTRUCCIÓN DEL DATAFRAME MAESTRO FINAL (ETL)
# Misión: Extraer, transformar y cargar todos los datos en un único
# fichero Parquet limpio, robusto y libre de look-ahead bias.
# ====================================================================
import pandas as pd
import sqlalchemy
import os
import numpy as np
from datetime import datetime

print("--- [INICIO DE LA CONSTRUCCIÓN DEL DataFrame Maestro Final] ---")

# --- 1. EXTRACCIÓN (EXTRACT) ---
print(" -> Paso 1: Extrayendo TODAS las tablas de la base de datos...")
try:
    DB_USER = "cryptonita_user"
    DB_PASSWORD = "TIZavoltio999"
    DB_HOST = "localhost"
    DB_PORT = "5432"
    DB_NAME = "cryptonita_db2"
    db_url = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    engine = sqlalchemy.create_engine(db_url)
    
    # Se leen todas las tablas, sin quitar ninguna
    tables_to_load = {
        'asset_metrics': "SELECT * FROM asset_metrics",
        'technical_indicators': "SELECT * FROM technical_indicators",
        'sentiment_metrics': "SELECT * FROM sentiment_metrics",
        'candle_patterns': "SELECT * FROM candle_patterns",
        'derivatives_funding_rates': "SELECT * FROM derivatives_funding_rates",
        'macro_spy': "SELECT timestamp, close AS spy_close FROM macro_spy",
        'macro_vix': "SELECT timestamp, close AS vix_close FROM macro_vix",
        'macro_tnx': "SELECT timestamp, close AS tnx_close FROM macro_tnx",
        'macro_dxy': "SELECT timestamp, close AS dxy_close FROM macro_dx_y_nyb",
        'macro_gc': "SELECT timestamp, close AS gc_close FROM macro_gc",
        'macro_cl': "SELECT timestamp, close AS cl_close FROM macro_cl",
    }
    
    dataframes = {name: pd.read_sql(query, engine) for name, query in tables_to_load.items()}
    print(" -> Tablas extraídas con éxito.")
    
except Exception as e:
    raise RuntimeError(f"❌ Error durante la extracción de datos: {e}")

# --- 2. TRANSFORMACIÓN (TRANSFORM) ---
print("\n -> Paso 2: Transformando y uniendo los datos...")

# 2.1: Asegurar tipos de datos de timestamp
for name, df in dataframes.items():
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True).dt.normalize()

# 2.2: Unir todas las tablas (sin excepciones)
master_df = dataframes['asset_metrics']
tables_by_ticker = ['technical_indicators', 'sentiment_metrics', 'candle_patterns', 'derivatives_funding_rates']
for name in tables_by_ticker:
    master_df = pd.merge(master_df, dataframes[name], on=['timestamp', 'ticker'], how='left')

tables_by_date = ['macro_spy', 'macro_vix', 'macro_tnx', 'macro_dxy', 'macro_gc', 'macro_cl']
for name in tables_by_date:
    master_df = pd.merge(master_df, dataframes[name], on='timestamp', how='left')

# 2.3: Manejo de nulos (lógica original completa)
print(" -> Gestionando valores nulos...")
macro_cols = ['spy_close', 'vix_close', 'tnx_close', 'dxy_close', 'gc_close', 'cl_close']
master_df.sort_values(by=['timestamp'], inplace=True)
master_df[macro_cols] = master_df[macro_cols].ffill()
master_df['sentiment_score'].fillna(0, inplace=True)
master_df['pattern_name'].fillna('no_pattern', inplace=True)
master_df['pattern_score'].fillna(0, inplace=True)
master_df['funding_rate'].fillna(0, inplace=True)

# 2.4: Ingeniería de características
print(" -> Calculando características financieras avanzadas...")
master_df.sort_values(by=['ticker', 'timestamp'], inplace=True)
def apply_feature_engineering(group):
    group['log_return'] = np.log(group['close'] / group['close'].shift(1))
    group['volatility_7d'] = group['log_return'].rolling(7).std()
    return group
master_df = master_df.groupby('ticker', group_keys=False).apply(apply_feature_engineering)

# 2.5: CORRECCIÓN DE LOOK-AHEAD BIAS
feature_cols = master_df.columns.drop(['timestamp', 'ticker', 'open', 'high', 'low', 'close', 'volume'])
print(" -> Aplicando retraso (shift)...")
master_df[feature_cols] = master_df.groupby('ticker')[feature_cols].shift(1)

# Limpieza final de NaNs
master_df.dropna(inplace=True)
master_df.reset_index(drop=True, inplace=True)
print(" -> Transformación completada.")

# --- 3. SANITY CHECK Y CARGA (LOAD) ---
print("\n -> Paso 3: Realizando comprobación de sanidad y guardando...")
assert master_df.isnull().sum().sum() == 0, "¡Alerta! Aún quedan valores nulos."
assert master_df.shape[0] > 100, f"¡Alerta! El DataFrame tiene muy pocas filas ({master_df.shape[0]})."
print("✅ Comprobación de sanidad superada.")

output_dir = 'dataframes/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- CAMBIO CLAVE: Nombre del archivo de salida ---
file_path = os.path.join(output_dir, 'master_df_new_universe.parquet')

master_df.to_parquet(file_path)

print(f"✅ ¡Éxito! Tu DataFrame Maestro para el NUEVO UNIVERSO ha sido guardado en: {file_path}")
print(f" -> Forma final del DataFrame: {master_df.shape}")
print("\n--- [FIN DE LA CONSTRUCCIÓN] ---")

--- [INICIO DE LA CONSTRUCCIÓN DEL DataFrame Maestro Final] ---
 -> Paso 1: Extrayendo TODAS las tablas de la base de datos...
 -> Tablas extraídas con éxito.

 -> Paso 2: Transformando y uniendo los datos...
 -> Gestionando valores nulos...
 -> Calculando características financieras avanzadas...
 -> Aplicando retraso (shift)...
 -> Transformación completada.

 -> Paso 3: Realizando comprobación de sanidad y guardando...


/var/folders/z_/rcq9bsy13fzbphk_82m0p03r0000gn/T/ipykernel_51120/4138816404.py:69: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  master_df['sentiment_score'].fillna(0, inplace=True)
/var/folders/z_/rcq9bsy13fzbphk_82m0p03r0000gn/T/ipykernel_51120/4138816404.py:70: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

AssertionError: ¡Alerta! El DataFrame tiene muy pocas filas (0).